In [1]:
import pandas as pd

data = pd.read_csv('data_sample_three_years.csv')

data = data.iloc[:, [0,1,2,3,4]]
data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.iloc[:, [0,5,1,2,3,4]]

data_backup = data.loc[ data.timestamp >= 1675814400 ]
data = data.loc[ data.timestamp < 1675814400 ]

from talib import RSI
data['rsi_14'] = RSI(data['close'], timeperiod=14)
#data['rsi_30'] = RSI(data['close'], timeperiod=30)

from talib import MA, SMA, EMA
data['ma_9'] = MA(data['close'], timeperiod=9)
#data['ma_52'] = MA(data['close'], timeperiod=52)

#for i in range(1,11):
#    data[f'high+{i}'] = data['high'].shift(i)
#for i in range(1,11):
#    data[f'low+{i}'] = data['low'].shift(i)
data.dropna(axis=0, inplace=True)

In [2]:
import numpy as np

data['signal'] = np.where((data['close'] > data['ma_9']) & (data['rsi_14'] > 60), 1, np.where((data['close'] < data['ma_9']) & (data['rsi_14'] < 40), -1, 0))

In [3]:
#for i, d in data.iterrows():
#    print(d['signal'])

In [4]:
inputs = data.iloc[:, [5,6,7]]
target = data.iloc[:, [8]]

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=1, shuffle=False)

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import xgboost

2024-03-04 05:49:44.095861: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-04 05:49:44.380273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 05:49:44.380363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 05:49:44.384834: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 05:49:44.462250: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-04 05:49:47.134978: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [7]:
tree = DecisionTreeClassifier(random_state=1)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=1)

In [8]:
# Make predictions on the testing data
y_pred = tree.predict(X_test)

# Evaluate the model using mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.07261538461538461


In [9]:
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [10]:
y_test['pred'] = y_pred

In [11]:
y_test

,signal,pred
13012,1,0
13013,0,0
13014,0,0
13015,0,0
13016,-1,-1
...,...,...
16257,0,0
16258,1,1
16259,1,1
16260,1,1


In [19]:
#y_test.to_excel('y_test.xlsx')

In [16]:
raw_data = data.iloc[12998:]

In [17]:
raw_data['pred'] = y_pred

/tmp/ipykernel_1446913/2607920888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['pred'] = y_pred


In [18]:
raw_data

,timestamp,date,open,high,low,close,rsi_14,ma_9,signal,pred
13012,1662915600,2022-09-11 17:00:00,1769.84,1787.88,1765.54,1770.22,60.210983,1765.906667,1,0
13013,1662919200,2022-09-11 18:00:00,1770.22,1775.65,1753.33,1758.85,52.541380,1765.955556,0,0
13014,1662922800,2022-09-11 19:00:00,1758.85,1763.90,1757.00,1761.90,54.225769,1766.081111,0,0
13015,1662926400,2022-09-11 20:00:00,1761.89,1766.66,1744.10,1757.69,51.508260,1764.858889,0,0
13016,1662930000,2022-09-11 21:00:00,1757.68,1764.36,1718.02,1731.25,38.469285,1760.370000,-1,-1
...,...,...,...,...,...,...,...,...,...,...
16257,1675792800,2023-02-07 18:00:00,1672.03,1679.03,1625.48,1638.52,51.870094,1637.107778,0,0
16258,1675796400,2023-02-07 19:00:00,1638.52,1658.48,1637.64,1656.42,63.059562,1639.923333,1,1
16259,1675800000,2023-02-07 20:00:00,1656.43,1668.77,1650.16,1664.93,66.988870,1643.332222,1,1
16260,1675803600,2023-02-07 21:00:00,1664.86,1674.72,1663.76,1667.43,68.063590,1647.513333,1,1


In [20]:
raw_data.to_excel('y_test.xlsx')